In [8]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import pymysql

import pandas as pd
import pymysql

# List of CSV file paths
csv_files = [
    "AP_bus_details.csv", "assam_bus_details.csv", "chandigarh_bus_details.csv",
    "himachal_bus_details.csv", "kaac_bus_details.csv", "kerala_bus_details.csv",
    "rajasthan_bus_details.csv", "sb_bus_details.csv", "Telangana_bus_details.csv",
    "up_bus_details.csv", "wb_bus_details.csv", "wb2_bus_details.csv", "jk_bus_details.csv"
]

# Read each CSV file into a DataFrame and store it in a list
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Add id column
combined_df.insert(0, 'id', range(1, len(combined_df) + 1))

# Save to a CSV file
combined_df.to_csv("bus_routes.csv", index=False)

# Read the CSV file again for further processing
df = pd.read_csv("bus_routes.csv")

# Replace string 'INR ' with empty string in 'Price' column
df['Price'] = df['Price'].str.replace('INR ', '', regex=False)

# Extract digits (0-9) from 'Seat_Availability' using a raw string
df['Seat_Availability'] = df['Seat_Availability'].str.extract(r'(\d+)')

# Handle NaN values before inserting into the database
df.fillna(value=0, inplace=True)  # You can choose a more appropriate value if needed

# Python MySQL connection
myconnection = pymysql.connect(host='localhost', user='root', passwd='1234', database="redbus")

# Create a cursor for executing SQL queries
cursor = myconnection.cursor()

# Create table in MySQL
column_types = ",".join(f"{i} {j}" for i, j in zip(df.columns, df.dtypes)).replace("float64", "FLOAT").replace("object", "TEXT").replace("int64", "INT")
table_name = "bus_routes"
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({column_types})"
cursor.execute(create_table_query)

# Insert data into MySQL table
insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"
for row in df.itertuples(index=False, name=None):
    cursor.execute(insert_query, row)

# Commit the changes
myconnection.commit()

# Close the cursor and connection
cursor.close()
myconnection.close()
